Just another version

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import os

from awesome_progress_bar import ProgressBar

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from utils import LoggerCallback, DataGenerator

In [2]:
df = pd.read_csv('data/train_base_2.csv', index_col=0)

def read(path):
    fp = np.memmap(path, dtype='float32', mode='r', shape=(316, 316, 1))
    
    directory, filename = os.path.split(path)
    n = int(filename.replace('.dat', ''))
    n /= len(glob.glob(f'{directory}/*.dat'))
    
    patient = directory.split('\\')[-1]
    
    meta = [
        n,
        df.loc[patient, 'Sex'], 
        df.loc[patient, 'Age'], 
        df.loc[patient, 'SmokingStatus'],
        df.loc[patient, 'FVC_0'],
        df.loc[patient, 'Week'],
    ]
    
    y = []
    for x in [6, 8, 10, 12, 18]:
        y.append(df.loc[patient, f'FVC_{x}'])
        
    return fp, meta, y

In [3]:
images = glob.glob('data\\train\\**/*.dcm')
target_size = (316, 316)

In [4]:
def MySeparableConv2D(n_units):
    def _(layer):
        layer = SeparableConv2D(n_units, 3, activation='relu', activity_regularizer=l2(0.6))(layer)
        layer = SeparableConv2D(n_units, 3, activation='relu', activity_regularizer=l2(0.6))(layer)
        return MaxPool2D()(layer)
    return _

def create_embedder():
    inp = Input([*target_size, 1])
    inp2 = Input([6])

    layer = MySeparableConv2D(32)(inp)
    layer = MySeparableConv2D(64)(layer)
    layer = MySeparableConv2D(128)(layer)
    layer = MySeparableConv2D(256)(layer)
    layer = GlobalAvgPool2D()(layer)
    
    layer2 = Dense(16, activation='relu')(inp2)

    layer = Concatenate()([layer, layer2])
    out = Dense(16)(layer)

    return Model([inp, inp2], out)

embedder = create_embedder()

In [5]:
layer = Dense(24, activation='relu')(embedder.output)
layer = Dense(5)(layer)
model = Model(embedder.input, layer)
model.compile('adam', 'mse', ['mape'])

In [6]:
test_size = 0.1

np.random.seed(42)

patients = os.listdir('data/train')
np.random.shuffle(patients)

train_size = int(np.round(len(patients) * (1 - test_size)))
patients_train = patients[:train_size]
patients_test = patients[train_size:]

dat_train = []
for x in patients_train:
    dat_train.extend(glob.glob(f'data\\train\\{x}/*.dat'))
dat_test = []
for x in patients_test:
    dat_test.extend(glob.glob(f'data\\train\\{x}/*.dat'))

print(f'Train dataset size: {len(dat_train)}')
print(f'Test dataset size: {len(dat_test)}')

Train dataset size: 30153
Test dataset size: 3475


In [7]:
%%time

train = DataGenerator(dat_train, read)
test = DataGenerator(dat_test, read)
tf.get_logger().setLevel("ERROR") 

history = model.fit(
    train, 
    validation_data=test,
    epochs=15,
    callbacks=[LoggerCallback(len(train))],
    verbose=0,
)

Epoch 0:
  941/941   |========================= 11:42 ==========================| 100.00%   mape:  66.89 - val_mape:  15.34
Epoch 1:
  941/941   |========================= 11:43 ==========================| 100.00%   mape:  51.09 - val_mape:  19.04
Epoch 2:
  941/941   |========================= 11:47 ==========================| 100.00%   mape:  48.99 - val_mape:  17.10
Epoch 3:
  941/941   |========================= 11:51 ==========================| 100.00%   mape:  47.62 - val_mape:  19.01
Epoch 4:
  941/941   |========================= 11:41 ==========================| 100.00%   mape:  46.20 - val_mape:  18.24
Epoch 5:
  941/941   |========================= 11:41 ==========================| 100.00%   mape:  44.79 - val_mape:  18.31
Epoch 6:
  941/941   |========================= 11:33 ==========================| 100.00%   mape:  43.43 - val_mape:  20.57
Epoch 7:
  941/941   |========================= 11:44 ==========================| 100.00%   mape:  42.38 - val_mape:  20.68
Epoch 8:

In [8]:
def embed_patient(directory):
    imgs = []
    meta = []
    
    dats = glob.glob(f'{directory}\\*.dat')
    dats.sort(key=lambda d: int(d[:-4].split('\\')[-1]))
    for dat in dats:
        img, m, _ = read(dat)
        imgs.append(img)
        meta.append(m)
    
    imgs = np.array(imgs)
    meta = np.array(meta)
    
    embedding = embedder.predict([imgs, meta])
        
    return np.hstack([
        np.min(embedding, axis=0),
        np.mean(embedding, axis=0),
        np.max(embedding, axis=0),
    ])

In [9]:
embeddings = []

patients = os.listdir('data\\train')
bar = ProgressBar(len(patients))
for patient in patients:
    bar.iter()
    embeddings.append(embed_patient(f'data\\train\\{patient}'))
    
embeddings = pd.DataFrame(embeddings, index=patients)

Progress:   |=============================== 03:11 ===============================| 100.00% Complete


In [10]:
new_df = pd.read_csv('data/train.csv', index_col=0)
new_df = new_df.drop(['Percent', 'Sex', 'Age', 'SmokingStatus'], axis=1)
new_df = pd.merge(new_df, embeddings, left_index=True, right_index=True)

In [16]:
df_train = new_df.loc[patients_train]
df_test = new_df.loc[patients_test]

In [19]:
from sklearn.preprocessing import StandardScaler

X_train = df_train.loc[:, new_df.columns != 'FVC']
X_test = df_test.loc[:, new_df.columns != 'FVC']
y_train = df_train.FVC
y_test = df_test.FVC

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
reg = RandomForestRegressor(random_state=0)
reg.fit(X_train, y_train)
reg.score(X_test, y_test)

0.8584843014422967